In [1]:
import os
import re
import glob
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

# XGBoost (opcional)
try:
    from xgboost import XGBClassifier
    xgb_available = True
except ImportError:
    xgb_available = False

In [2]:
DATA_DIR = "/Users/rodrigokauer/Desktop/tcc/dataset/footystats" 
WINDOW_SIZE = 5
H2H_WINDOW = 5
RETA_FINAL_TH = 0.75
RANDOM_STATE = 42

In [3]:

print("Carregando FootyStats...")
paths = sorted(glob.glob(os.path.join(DATA_DIR, "brazil-serie-a-matches-*-stats.csv")))
if not paths:
    raise FileNotFoundError(f"Nenhum arquivo encontrado em: {DATA_DIR}")

dfs = []
for p in paths:
    tmp = pd.read_csv(p)
    m = re.search(r"matches-(\d{4})-to-(\d{4})", os.path.basename(p))
    tmp["season"] = int(m.group(1)) if m else np.nan
    tmp["source_file"] = os.path.basename(p)
    dfs.append(tmp)

df = pd.concat(dfs, ignore_index=True)

print("Shape:", df.shape)
print("Exemplo de colunas (primeiras 40):")
print(list(df.columns[:40]))



Carregando FootyStats...
Shape: (4560, 68)
Exemplo de colunas (primeiras 40):
['timestamp', 'date_GMT', 'status', 'attendance', 'home_team_name', 'away_team_name', 'referee', 'Game Week', 'Pre-Match PPG (Home)', 'Pre-Match PPG (Away)', 'home_ppg', 'away_ppg', 'home_team_goal_count', 'away_team_goal_count', 'total_goal_count', 'total_goals_at_half_time', 'home_team_goal_count_half_time', 'away_team_goal_count_half_time', 'home_team_goal_timings', 'away_team_goal_timings', 'home_team_corner_count', 'away_team_corner_count', 'home_team_yellow_cards', 'home_team_red_cards', 'away_team_yellow_cards', 'away_team_red_cards', 'home_team_first_half_cards', 'home_team_second_half_cards', 'away_team_first_half_cards', 'away_team_second_half_cards', 'home_team_shots', 'away_team_shots', 'home_team_shots_on_target', 'away_team_shots_on_target', 'home_team_shots_off_target', 'away_team_shots_off_target', 'home_team_fouls', 'away_team_fouls', 'home_team_possession', 'away_team_possession']


In [4]:
def pick_first(existing_cols, candidates):
    for c in candidates:
        if c in existing_cols:
            return c
    return None

cols = set(df.columns)

# candidatos típicos do FootyStats
home_team_col = pick_first(cols, ["home_team_name", "home_team", "Home Team", "homeTeam", "team_home"])
away_team_col = pick_first(cols, ["away_team_name", "away_team", "Away Team", "awayTeam", "team_away"])

home_goals_col = pick_first(cols, ["home_goal_count", "home_goals", "home_score", "home_team_goal_count", "homeTeamGoals"])
away_goals_col = pick_first(cols, ["away_goal_count", "away_goals", "away_score", "away_team_goal_count", "awayTeamGoals"])

date_col = pick_first(cols, ["date_GMT", "date", "match_date", "Date"])

if not all([home_team_col, away_team_col, home_goals_col, away_goals_col, date_col]):
    print("\n[ERRO] Não consegui encontrar automaticamente as colunas essenciais.")
    print("home_team_col:", home_team_col)
    print("away_team_col:", away_team_col)
    print("home_goals_col:", home_goals_col)
    print("away_goals_col:", away_goals_col)
    print("date_col:", date_col)
    print("\nColunas disponíveis:")
    print(sorted(list(df.columns)))
    raise KeyError("Colunas essenciais não encontradas. Veja a lista e ajuste os nomes candidatos.")

# padronizar nomes canônicos
df = df.rename(columns={
    home_team_col: "home_team_name",
    away_team_col: "away_team_name",
    home_goals_col: "home_goals",
    away_goals_col: "away_goals",
    date_col: "date_GMT"
})

df["date_GMT"] = pd.to_datetime(df["date_GMT"], errors="coerce")
df = df.sort_values("date_GMT").reset_index(drop=True)

/var/folders/s6/4g4mk68x1679121_jq2y5hwh0000gn/T/ipykernel_88036/700047318.py:38: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["date_GMT"] = pd.to_datetime(df["date_GMT"], errors="coerce")


In [5]:
odds_cols = [c for c in df.columns if "odds" in c.lower()]
xg_cols = [c for c in df.columns if re.search(r"\bxg\b|expected", c, re.I)]
for c in sorted(set(odds_cols + xg_cols)):
    df[c] = df[c].replace(0, np.nan)

In [6]:
df["target"] = np.select(
    [df["home_goals"] > df["away_goals"],
     df["home_goals"] == df["away_goals"],
     df["home_goals"] < df["away_goals"]],
    [2, 1, 0]
)

In [7]:
df["rodada"] = df.groupby("season").cumcount() + 1
df["rodadas_total"] = df.groupby("season")["rodada"].transform("max")
df["fase_campeonato"] = df["rodada"] / df["rodadas_total"]
df["reta_final"] = (df["fase_campeonato"] >= RETA_FINAL_TH).astype(int)

In [8]:
TEAM_STATE = {
    "Flamengo":"RJ","Fluminense":"RJ","Vasco":"RJ","Vasco da Gama":"RJ","Botafogo":"RJ",
    "Palmeiras":"SP","Corinthians":"SP","São Paulo":"SP","Sao Paulo":"SP","Santos":"SP",
    "Cruzeiro":"MG","Atlético Mineiro":"MG","Atletico Mineiro":"MG","Atlético-MG":"MG",
    "Grêmio":"RS","Gremio":"RS","Internacional":"RS",
    "Athletico Paranaense":"PR","Athletico-PR":"PR","Coritiba":"PR",
    "Bahia":"BA","Vitória":"BA","Vitoria":"BA",
    "Avaí":"SC","Avai":"SC","Figueirense":"SC","Chapecoense":"SC",
    "Ceará":"CE","Ceara":"CE","Fortaleza":"CE",
    "Sport":"PE","Sport Recife":"PE","Náutico":"PE","Nautico":"PE","Santa Cruz":"PE",
    "Goiás":"GO","Goias":"GO","Atlético Goianiense":"GO","Atletico Goianiense":"GO","Atlético-GO":"GO",
    "CSA":"AL","CRB":"AL",
    "Cuiabá":"MT","Cuiaba":"MT"
}

df["uf_home"] = df["home_team_name"].map(TEAM_STATE)
df["uf_away"] = df["away_team_name"].map(TEAM_STATE)
df["confronto_regional"] = np.where(
    df["uf_home"].notna() & df["uf_away"].notna(),
    (df["uf_home"] == df["uf_away"]).astype(int),
    np.nan
)

In [9]:
main_odds = ["odds_ft_home_team_win", "odds_ft_draw", "odds_ft_away_team_win"]
if all(c in df.columns for c in main_odds):
    df["p_home_raw"] = 1.0 / df["odds_ft_home_team_win"]
    df["p_draw_raw"] = 1.0 / df["odds_ft_draw"]
    df["p_away_raw"] = 1.0 / df["odds_ft_away_team_win"]
    s = df[["p_home_raw", "p_draw_raw", "p_away_raw"]].sum(axis=1)
    df["p_home"] = df["p_home_raw"] / s
    df["p_draw"] = df["p_draw_raw"] / s
    df["p_away"] = df["p_away_raw"] / s
    df["p_gap_home_away"] = df["p_home"] - df["p_away"]
    df["odds_gap_home_away"] = df["odds_ft_away_team_win"] - df["odds_ft_home_team_win"]
else:
    print("[AVISO] Odds 1X2 não encontradas. Pipeline seguirá sem essas features.")


In [10]:
print("Calculando H2H...")

base_h2h = df[["date_GMT", "home_team_name", "away_team_name", "home_goals", "away_goals"]].copy()

def h2h_features(row, window=H2H_WINDOW):
    a, b, dt = row["home_team_name"], row["away_team_name"], row["date_GMT"]

    mask = (
        (((base_h2h["home_team_name"] == a) & (base_h2h["away_team_name"] == b)) |
         ((base_h2h["home_team_name"] == b) & (base_h2h["away_team_name"] == a)))
        & (base_h2h["date_GMT"] < dt)
    )

    hist = base_h2h.loc[mask].sort_values("date_GMT").tail(window)

    if hist.empty:
        return pd.Series({"h2h_games":0,"h2h_home_wins":0,"h2h_away_wins":0,"h2h_draws":0,"h2h_goal_diff":0})

    win_home=win_away=draws=saldo=0
    for _, r in hist.iterrows():
        # gols do time A (mandante atual)
        if r["home_team_name"] == a:
            ga, gb = r["home_goals"], r["away_goals"]
        else:
            ga, gb = r["away_goals"], r["home_goals"]

        saldo += (ga - gb)
        if ga > gb: win_home += 1
        elif ga < gb: win_away += 1
        else: draws += 1

    return pd.Series({"h2h_games":len(hist),"h2h_home_wins":win_home,"h2h_away_wins":win_away,"h2h_draws":draws,"h2h_goal_diff":saldo})

df = pd.concat([df, df.apply(h2h_features, axis=1)], axis=1)

Calculando H2H...


In [11]:
print("Construindo rolling por time...")

def safe(name, default=np.nan):
    return df[name] if name in df.columns else default

def build_team_view(home=True):
    if home:
        team = df["home_team_name"]; opp = df["away_team_name"]
        gf = df["home_goals"]; ga = df["away_goals"]
        shots = safe("home_shots"); sot = safe("home_shots_on_target")
        poss = safe("home_possession")
        corners = safe("home_corner_count")
        yellow = safe("home_yellow_cards"); red = safe("home_red_cards")
        fouls = safe("home_fouls")
        xg = safe("home_team_xg")
    else:
        team = df["away_team_name"]; opp = df["home_team_name"]
        gf = df["away_goals"]; ga = df["home_goals"]
        shots = safe("away_shots"); sot = safe("away_shots_on_target")
        poss = safe("away_possession")
        corners = safe("away_corner_count")
        yellow = safe("away_yellow_cards"); red = safe("away_red_cards")
        fouls = safe("away_fouls")
        xg = safe("away_team_xg")

    out = pd.DataFrame({
        "date_GMT": df["date_GMT"],
        "season": df["season"],
        "team": team,
        "opponent": opp,
        "gols_pro": gf,
        "gols_sofridos": ga,
        "chutes": shots,
        "chutes_no_alvo": sot,
        "posse_de_bola": poss,
        "escanteios": corners,
        "cartao_amarelo": yellow,
        "cartao_vermelho": red,
        "faltas": fouls,
        "xg": xg,
        "is_home": int(home),
        "target": df["target"]
    })

    out["pontos"] = np.select(
        [out["gols_pro"] > out["gols_sofridos"], out["gols_pro"] == out["gols_sofridos"]],
        [3, 1], default=0
    )
    out["win"] = (out["pontos"] == 3).astype(int)
    out["draw"] = (out["pontos"] == 1).astype(int)
    out["loss"] = (out["pontos"] == 0).astype(int)
    out["forma1"] = np.select([out["win"]==1, out["draw"]==1, out["loss"]==1],[1,0,-1], default=0)
    return out

home_df = build_team_view(True)
away_df = build_team_view(False)
team_df = pd.concat([home_df, away_df]).sort_values(["team","date_GMT"]).reset_index(drop=True)

features_to_roll = ["gols_pro","gols_sofridos","pontos","chutes","chutes_no_alvo","posse_de_bola",
                    "escanteios","cartao_amarelo","cartao_vermelho","faltas","xg","win","draw","loss","forma1"]

roll_all = (
    team_df.groupby("team")[features_to_roll]
    .apply(lambda x: x.rolling(window=WINDOW_SIZE, min_periods=1).mean().shift(1))
    .reset_index(level=0, drop=True)
)
roll_all.columns = [f"media_{c}_ult_{WINDOW_SIZE}" for c in roll_all.columns]
team_df = pd.concat([team_df, roll_all], axis=1)

roll_side = (
    team_df.groupby(["team","is_home"])[features_to_roll]
    .apply(lambda x: x.rolling(window=WINDOW_SIZE, min_periods=1).mean().shift(1))
    .reset_index(level=[0,1], drop=True)
)
roll_side.columns = [f"media_side_{c}_ult_{WINDOW_SIZE}" for c in roll_side.columns]
team_df = pd.concat([team_df, roll_side], axis=1)

Construindo rolling por time...


In [12]:
print("Montando df_final...")

base_cols = ["date_GMT","season","home_team_name","away_team_name","target",
             "rodada","fase_campeonato","reta_final","confronto_regional",
             "h2h_games","h2h_home_wins","h2h_away_wins","h2h_draws","h2h_goal_diff"]

# odds derivadas (se existirem)
for c in ["p_home","p_draw","p_away","p_gap_home_away","odds_gap_home_away"]:
    if c in df.columns:
        base_cols.append(c)

df_match = df[base_cols].copy()

team_feats = team_df[["date_GMT","team","is_home"] + [c for c in team_df.columns if c.startswith("media_") or c.startswith("media_side_")]].copy()

df_final = df_match.merge(
    team_feats[team_feats["is_home"] == 1],
    left_on=["date_GMT","home_team_name"],
    right_on=["date_GMT","team"],
    how="left"
).drop(columns=["team","is_home"])

rename_home = {c: f"home_{c}" for c in team_feats.columns if c.startswith("media_") or c.startswith("media_side_")}
df_final.rename(columns=rename_home, inplace=True)

df_final = df_final.merge(
    team_feats[team_feats["is_home"] == 0],
    left_on=["date_GMT","away_team_name"],
    right_on=["date_GMT","team"],
    how="left",
    suffixes=("", "_tmp")
).drop(columns=["team","is_home"])

rename_away = {c: f"away_{c}" for c in team_feats.columns if c.startswith("media_") or c.startswith("media_side_")}
df_final.rename(columns=rename_away, inplace=True)

df_final = df_final.sort_values("date_GMT").reset_index(drop=True)

Montando df_final...


In [13]:
print("\nPreparando modelagem MULTICLASSE (3 classes)...")

le = LabelEncoder()
le.fit(pd.concat([df_final["home_team_name"], df_final["away_team_name"]]).astype(str))

df_final["home_code"] = le.transform(df_final["home_team_name"].astype(str))
df_final["away_code"] = le.transform(df_final["away_team_name"].astype(str))

feature_cols = ["home_code","away_code","rodada","fase_campeonato","reta_final","confronto_regional",
                "h2h_games","h2h_home_wins","h2h_away_wins","h2h_draws","h2h_goal_diff"]

for c in ["p_home","p_draw","p_away","p_gap_home_away","odds_gap_home_away"]:
    if c in df_final.columns:
        feature_cols.append(c)

feature_cols += [c for c in df_final.columns if c.startswith("home_media_") or c.startswith("away_media_")]

X = df_final[feature_cols]
y = df_final["target"]

split = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split], X.iloc[split:]
y_train, y_test = y.iloc[:split], y.iloc[split:]

print("\nDistribuição de classes:")
print("Treino:\n", y_train.value_counts().sort_index())
print("Teste:\n", y_test.value_counts().sort_index())

models = {
    "Logistic Regression": LogisticRegression(max_iter=2000, random_state=RANDOM_STATE),
    "Random Forest": RandomForestClassifier(n_estimators=400, max_depth=12, random_state=RANDOM_STATE),
    "Gradient Boosting": GradientBoostingClassifier(random_state=RANDOM_STATE),
    "AdaBoost": AdaBoostClassifier(random_state=RANDOM_STATE),
    "Naive Bayes": GaussianNB(),
    "SVM": SVC(random_state=RANDOM_STATE),
    "KNN": KNeighborsClassifier(n_neighbors=20),
    "MLP": MLPClassifier(hidden_layer_sizes=(64,64), max_iter=800, random_state=RANDOM_STATE),
}

if xgb_available:
    models["XGBoost"] = XGBClassifier(
        objective="multi:softprob",
        num_class=3,
        eval_metric="mlogloss",
        random_state=RANDOM_STATE,
        n_estimators=500,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.9,
        colsample_bytree=0.9
    )

print("\nTreinando modelos (MULTICLASSE)...")
results = []
reports = {}

for name, model in models.items():
    print(f"Treinando {name}...")

    pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
        ("clf", model)
    ])

    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    prec_macro = precision_score(y_test, y_pred, average="macro", zero_division=0)
    rec_macro = recall_score(y_test, y_pred, average="macro", zero_division=0)
    f1_macro = f1_score(y_test, y_pred, average="macro", zero_division=0)

    prec_w = precision_score(y_test, y_pred, average="weighted", zero_division=0)
    rec_w = recall_score(y_test, y_pred, average="weighted", zero_division=0)
    f1_w = f1_score(y_test, y_pred, average="weighted", zero_division=0)

    results.append({
        "Modelo": name,
        "Acurácia": acc,
        "Precisão (macro)": prec_macro,
        "Recall (macro)": rec_macro,
        "F1 (macro)": f1_macro,
        "Precisão (weighted)": prec_w,
        "Recall (weighted)": rec_w,
        "F1 (weighted)": f1_w
    })

    reports[name] = classification_report(
        y_test, y_pred,
        labels=[0,1,2],
        target_names=["Visitante vence (0)","Empate (1)","Mandante vence (2)"],
        zero_division=0
    )

results_df = pd.DataFrame(results).sort_values(by="F1 (weighted)", ascending=False)
print("\n=== RANKING DE MODELOS (ordenado por F1 weighted) ===")
print(results_df)

best = results_df.iloc[0]["Modelo"]
print(f"\n=== CLASSIFICATION REPORT: {best} ===")
print(reports[best])


Preparando modelagem MULTICLASSE (3 classes)...

Distribuição de classes:
Treino:
 target
0     880
1     986
2    1782
Name: count, dtype: int64
Teste:
 target
0    248
1    239
2    425
Name: count, dtype: int64

Treinando modelos (MULTICLASSE)...
Treinando Logistic Regression...
Treinando Random Forest...


/Users/rodrigokauer/Desktop/tcc/.venv/lib/python3.9/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['home_media_chutes_ult_5' 'home_media_chutes_no_alvo_ult_5'
 'home_media_posse_de_bola_ult_5' 'home_media_escanteios_ult_5'
 'home_media_cartao_amarelo_ult_5' 'home_media_cartao_vermelho_ult_5'
 'home_media_faltas_ult_5' 'home_media_xg_ult_5'
 'home_media_side_chutes_ult_5' 'home_media_side_chutes_no_alvo_ult_5'
 'home_media_side_posse_de_bola_ult_5' 'home_media_side_escanteios_ult_5'
 'home_media_side_cartao_amarelo_ult_5'
 'home_media_side_cartao_vermelho_ult_5' 'home_media_side_faltas_ult_5'
 'home_media_side_xg_ult_5' 'away_media_chutes_ult_5'
 'away_media_chutes_no_alvo_ult_5' 'away_media_posse_de_bola_ult_5'
 'away_media_escanteios_ult_5' 'away_media_cartao_amarelo_ult_5'
 'away_media_cartao_vermelho_ult_5' 'away_media_faltas_ult_5'
 'away_media_xg_ult_5' 'away_media_side_chutes_ult_5'
 'away_media_side_chutes_no_alvo_ult_5'
 

Treinando Gradient Boosting...


/Users/rodrigokauer/Desktop/tcc/.venv/lib/python3.9/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['home_media_chutes_ult_5' 'home_media_chutes_no_alvo_ult_5'
 'home_media_posse_de_bola_ult_5' 'home_media_escanteios_ult_5'
 'home_media_cartao_amarelo_ult_5' 'home_media_cartao_vermelho_ult_5'
 'home_media_faltas_ult_5' 'home_media_xg_ult_5'
 'home_media_side_chutes_ult_5' 'home_media_side_chutes_no_alvo_ult_5'
 'home_media_side_posse_de_bola_ult_5' 'home_media_side_escanteios_ult_5'
 'home_media_side_cartao_amarelo_ult_5'
 'home_media_side_cartao_vermelho_ult_5' 'home_media_side_faltas_ult_5'
 'home_media_side_xg_ult_5' 'away_media_chutes_ult_5'
 'away_media_chutes_no_alvo_ult_5' 'away_media_posse_de_bola_ult_5'
 'away_media_escanteios_ult_5' 'away_media_cartao_amarelo_ult_5'
 'away_media_cartao_vermelho_ult_5' 'away_media_faltas_ult_5'
 'away_media_xg_ult_5' 'away_media_side_chutes_ult_5'
 'away_media_side_chutes_no_alvo_ult_5'
 

Treinando AdaBoost...


...

/Users/rodrigokauer/Desktop/tcc/.venv/lib/python3.9/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['home_media_chutes_ult_5' 'home_media_chutes_no_alvo_ult_5'
 'home_media_posse_de_bola_ult_5' 'home_media_escanteios_ult_5'
 'home_media_cartao_amarelo_ult_5' 'home_media_cartao_vermelho_ult_5'
 'home_media_faltas_ult_5' 'home_media_xg_ult_5'
 'home_media_side_chutes_ult_5' 'home_media_side_chutes_no_alvo_ult_5'
 'home_media_side_posse_de_bola_ult_5' 'home_media_side_escanteios_ult_5'
 'home_media_side_cartao_amarelo_ult_5'
 'home_media_side_cartao_vermelho_ult_5' 'home_media_side_faltas_ult_5'
 'home_media_side_xg_ult_5' 'away_media_chutes_ult_5'
 'away_media_chutes_no_alvo_ult_5' 'away_media_posse_de_bola_ult_5'
 'away_media_escanteios_ult_5' 'away_media_cartao_amarelo_ult_5'
 'away_media_cartao_vermelho_ult_5' 'away_media_faltas_ult_5'
 'away_media_xg_ult_5' 'away_media_side_chutes_ult_5'
 'away_media_side_chutes_no_alvo_ult_5'
 

Treinando KNN...
Treinando MLP...


/Users/rodrigokauer/Desktop/tcc/.venv/lib/python3.9/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['home_media_chutes_ult_5' 'home_media_chutes_no_alvo_ult_5'
 'home_media_posse_de_bola_ult_5' 'home_media_escanteios_ult_5'
 'home_media_cartao_amarelo_ult_5' 'home_media_cartao_vermelho_ult_5'
 'home_media_faltas_ult_5' 'home_media_xg_ult_5'
 'home_media_side_chutes_ult_5' 'home_media_side_chutes_no_alvo_ult_5'
 'home_media_side_posse_de_bola_ult_5' 'home_media_side_escanteios_ult_5'
 'home_media_side_cartao_amarelo_ult_5'
 'home_media_side_cartao_vermelho_ult_5' 'home_media_side_faltas_ult_5'
 'home_media_side_xg_ult_5' 'away_media_chutes_ult_5'
 'away_media_chutes_no_alvo_ult_5' 'away_media_posse_de_bola_ult_5'
 'away_media_escanteios_ult_5' 'away_media_cartao_amarelo_ult_5'
 'away_media_cartao_vermelho_ult_5' 'away_media_faltas_ult_5'
 'away_media_xg_ult_5' 'away_media_side_chutes_ult_5'
 'away_media_side_chutes_no_alvo_ult_5'
 

Treinando XGBoost...

=== RANKING DE MODELOS (ordenado por F1 weighted) ===
                Modelo  Acurácia  Precisão (macro)  Recall (macro)  \
4          Naive Bayes  0.479167          0.435123        0.439779   
8              XGBoost  0.489035          0.451702        0.417015   
2    Gradient Boosting  0.502193          0.443484        0.409387   
1        Random Forest  0.495614          0.423538        0.398017   
6                  KNN  0.475877          0.402873        0.383691   
3             AdaBoost  0.487939          0.307060        0.409475   
0  Logistic Regression  0.493421          0.396021        0.394058   
7                  MLP  0.405702          0.376773        0.375668   
5                  SVM  0.497807          0.393952        0.386967   

   F1 (macro)  Precisão (weighted)  Recall (weighted)  F1 (weighted)  
4    0.422179             0.463569           0.479167       0.459026  
8    0.408841             0.465410           0.489035       0.451608  
2    0.377

/Users/rodrigokauer/Desktop/tcc/.venv/lib/python3.9/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['home_media_chutes_ult_5' 'home_media_chutes_no_alvo_ult_5'
 'home_media_posse_de_bola_ult_5' 'home_media_escanteios_ult_5'
 'home_media_cartao_amarelo_ult_5' 'home_media_cartao_vermelho_ult_5'
 'home_media_faltas_ult_5' 'home_media_xg_ult_5'
 'home_media_side_chutes_ult_5' 'home_media_side_chutes_no_alvo_ult_5'
 'home_media_side_posse_de_bola_ult_5' 'home_media_side_escanteios_ult_5'
 'home_media_side_cartao_amarelo_ult_5'
 'home_media_side_cartao_vermelho_ult_5' 'home_media_side_faltas_ult_5'
 'home_media_side_xg_ult_5' 'away_media_chutes_ult_5'
 'away_media_chutes_no_alvo_ult_5' 'away_media_posse_de_bola_ult_5'
 'away_media_escanteios_ult_5' 'away_media_cartao_amarelo_ult_5'
 'away_media_cartao_vermelho_ult_5' 'away_media_faltas_ult_5'
 'away_media_xg_ult_5' 'away_media_side_chutes_ult_5'
 'away_media_side_chutes_no_alvo_ult_5'
 